In [1]:
import os,csv
import numpy as np
import tensorflow as tf
import keras
from mtcnn import MTCNN
from numpy import asarray
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image 
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from PIL import Image
from scipy.spatial.distance import euclidean,cosine


image_path1 = 'gt_db/s01/01.jpg'
image_path2 = 'gt_db/s02/01.jpg'


In [2]:
def extract_face(filename, required_size=(224, 224)):
    
    pixels = plt.imread(filename)
    
    detector = MTCNN()
	
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
	
    face = pixels[y1:y2, x1:x2]
	
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    face_array = face_array.astype('float32')
    
    img_array = np.expand_dims(face_array, axis=0)
    img_array = preprocess_input(img_array, version=2)
    
    return img_array

    


In [4]:
def is_Match(known_feat, candidate_feat, threshold=0.5):
    score = cosine(known_feat, candidate_feat)
    if score <= threshold:
        print('face is a Match (%.3f <= %.3f)' % (score, threshold))
    else:
        print('face is NOT a Match (%.3f > %.3f)' % (score, threshold))



#### TEST KHFIF

In [3]:

img_array1 = extract_face(image_path1)
img_array2 = extract_face(image_path2)

# Charger le modèle SENet50 pré-entraîné
model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Extraire les features de l'image
features1 = model.predict(img_array1)
features2 = model.predict(img_array2)

# Afficher les features extraites
print("Features shape:", features1.shape)
print("Features:", features1.flatten())
print("Features shape:", features2.shape)
print("Features:", features2.flatten())

hashed_features1 = hash_features(features1, output_dim=64)
hashed_features2 = hash_features(features2, output_dim=64)

# Print the size and content of the hashed feature vector
print("Hashed features shape:", hashed_features1.shape)
print("Hashed features:", hashed_features1[0])
print("Hashed features shape:", hashed_features2.shape)
print("Hashed features:", hashed_features2)

# DISTANCES

is_match = is_Match(features1.flatten(),features2.flatten())
# is_match1 = is_Match(hashed_features1,hashed_features2)


1/1 [==============================] - 0s 97ms/step
Features shape: (1, 2048)
Features: [ 0.02077088  9.996358    2.117076   ...  0.02352208  0.4181638
 11.490126  ]
Features shape: (1, 2048)
Features: [0.02805781 0.0024663  0.0023163  ... 0.00627308 0.10347446 0.19576067]


NameError: name 'hash_features' is not defined

In [4]:
# Function to save hashed features to a CSV file with image names and hash codes in separate columns
def save_hashed_features_to_csv(csv_file, img_name, hashed_features,name,age,job):
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        features_str = np.array2string(hashed_features, separator=',', threshold=np.inf)  # Remove brackets
        writer.writerow([f"{img_name}",name, age, job,features_str])

In [5]:
names = [
    "Abdelaziz", "Abdeljalil", "Karim", "Abdelkhalek", "Aziza", "Abdelmajid", "Aymane", "Abdelmounaim", "Abdelouahab",
    "Sara", "Abdelrazak", "Abderrahim", "Abderrazak", "Abdeslam", "Nour", "Abir", "Achraf", "Adil", "Afif", "Ahmed",
    "Salma", "Akram", "Alaeddine", "Alae", "Ali", "Sanae", "Anas", "Anouar", "Assem", "Boutaina",
    "Aziz", "Badr", "Bahir", "Bashir", "Bilal", "Chafik", "Chakib", "Driss", "Elhadi", "Elmehdi",
    "Faouzi", "Fouad", "karima", "Hamid", "Hamza", "Hassan", "Hatim", "Hicham", "Idriss", "Ilyas","MAROUAN CHOTTA","YASSINE OUTSILA","OUSSAMA ETTAOUIL","YASSINE BOUHRAMACHE"
]

jobs= [
    "Project Manager","Business Analyst","Operations Manager","Marketing Manager","Sales Representative","Customer Service Manager","Financial Analyst",
    "Human Resources Coordinator","Product Developer","Logistics Specialist","Supply Chain Manager","Quality Control Inspector",
    "Research Scientist","Regulatory Affairs Specialist","Compliance Officer","Data Analyst","Operations Coordinator",
    "Inventory Manager","Procurement Specialist","Production Supervisor","Environmental Health and Safety Manager",
    "Industrial Engineer","Maintenance Technician","Packaging Designer","Technical Support Specialist","IT Systems Administrator","Software Developer",
    "Network Engineer","Cybersecurity Analyst","UX/UI Designer","Content Manager","Brand Strategist","Social Media Coordinator",
    "Event Planner","Public Relations Specialist","Legal Counsel","Risk Manager","Business Development Manager","Project Coordinator","Training Specialist",
    "Health and Wellness Coordinator","Account Manager","Financial Planner","Customer Success Manager","Sales Engineer","Operations Analyst",
    "Procurement Manager","Quality Assurance Engineer","Research Analyst","Regulatory Compliance Manager","DATA SCIENTIST","DATA SCIENTIST","DATA SCIENTIST","DATA SCIENTIST"
]

ages = [
    33, 26, 27, 25, 62, 30, 23, 28, 33, 26,
    25, 29, 37, 24, 39, 34, 41, 23, 27, 37,
    35, 35, 26, 36, 40, 26, 24, 31, 32, 25,
    34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
    44, 45, 25, 26, 27, 28, 29, 30, 29, 41,
    21,21,21,24
]





In [6]:
import random
def process_images_in_directory(root_dir,csv_file):
    i=0
    model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    for folder_name in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder_name)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(folder_path, file)
                    try:
                        # Load and preprocess the image
                        img_array = extract_face(img_path)
                        # Extract features from the image
                        features = model.predict(img_array)
                        # Hash the feature vector to size 64
                        # hashed_features = hash_features(features, output_dim=64)
                        # Save the hashed features to the CSV file with folder name + image name and hash codes
                        save_hashed_features_to_csv(csv_file, img_path, features.flatten(),names[i],ages[i],jobs[i])
                    
                    except Exception as e:
                        print(f"Failed to process {img_path}: {e}")
            i=i+1


root_directory = 'gt_db'  

csv_file = 'hashed_features1.csv'  # Replace with the desired path for the CSV file

process_images_in_directory(root_directory,csv_file)


1/1 [==============================] - 0s 365ms/step


KeyboardInterrupt: 

In [9]:
csv_file = 'hashed_features.csv'  # Replace with the desired path for the CSV file
save_hashed_features_to_csv(csv_file, image_path, hashed_features1,'ahmed','ali','nor')
 

NameError: name 'image_path' is not defined

In [ ]:
# np.set_printoptions(threshold=np.inf)  # Ensure that all values are printed

dirName = 'gt_db'
    
# Get the list of all files in directory tree at given path
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(dirName):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]

In [ ]:
def get_features(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]	
	samples = asarray(faces, 'float32')

	model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	
	yhat = model.predict(samples)
	return yhat

features = get_features(listOfFiles[0:2])


1/1 [==============================] - 0s 170ms/step


ValueError: in user code:

    File "c:\Users\LENOVO\Desktop\Face_recognition\face_reco\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\LENOVO\Desktop\Face_recognition\face_reco\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\LENOVO\Desktop\Face_recognition\face_reco\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\LENOVO\Desktop\Face_recognition\face_reco\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\LENOVO\Desktop\Face_recognition\face_reco\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\LENOVO\Desktop\Face

In [ ]:
listOfFiles[0]

'gt_db\\s01\\01.jpg'